In [7]:
import json
import requests
import pymongo
import boto3
import os
import datetime

In [26]:
# Setting environmental variables to be used in lambda

DATABASE_NAME = os.getenv("DATABASE_NAME"),
COLLECTION_NAME = os.getenv("COLLECTION_NAME")
MONGODB_URI = os.getenv("MONGODB_URI")
S3BUCKET_NAME = os.getenv("BUCKET_NAME")

In [6]:
DATA_SOURCE_URL = f"https://www.consumerfinance.gov/data-research/consumer-complaints/search/api/v1/" \
                  f"?field=all&format=json" \
                  f"&date_received_max=<TO_DATE>&date_received_min=<FROM_DATE>"

In [11]:
FROM_DATE = "2023-01-01"
FROM_DATE = datetime.datetime.strptime(FROM_DATE, "%Y-%m-%d")
FROM_DATE

datetime.datetime(2023, 1, 1, 0, 0)

In [ ]:
client = pymongo.MongoClient(MONGODB_URI)

In [17]:
def get_from_date_to_date():
    FROM_DATE = "2023-01-01"
    FROM_DATE = datetime.datetime.strptime(FROM_DATE, "%Y-%m-%d")

    if COLLECTION_NAME in client[DATABASE_NAME].list_collection_names():
        # To query the document in descending order to extract the document with latest to_date 
        latest_document = client[DATABASE_NAME][COLLECTION_NAME].find_one(sort=[("to_date", -1)]) 

        if latest_document is not None:
            FROM_DATE = latest_document["to_date"]

    TO_DATE = datetime.datetime.now()   # Assigning current date to TO_DATE

    response = {
        "from_Date" : FROM_DATE.strftime("%Y-%m-%d"),
        "to_date" : TO_DATE.strftime("%Y-%m-%d"),
        "from_date_object" : FROM_DATE,
        "to_date_object" : TO_DATE
    }
    return response

NameError: name 'client' is not defined

In [ ]:
def ingest_from_date_to_date(data, status):
    data.update("status" : status)
    client[DATABASE_NAME][COLLECTION_NAME].insert_one(data)

In [ ]:
def lambda_handler(event, context):
    print(event,context)
    from_date, to_date, from_date_object, to_date_object = get_from_date_to_date().values()
    if to_date == from_date:
        return {
            'statusCode': 200,
            'body': json.dumps(f"Pipeline has already downloaded all data upto {from_date}")
        }
    url = DATA_SOURCE_URL.replace("<todate>", to_date).replace("<fromdate>", from_date)
    data = requests.get(url)
    json_data = json.loads(data.content)

    def extract_source_data_from_url(json_data):
        _source_data = []
        for item in json_data:
            if "_source" in item.keys():
                _source_data.append(item["_source"])
        return _source_data

    consumer_complaints_data = extract_source_data_from_url(json_data)

    s3 = boto3.resource('s3')
    s3object = s3.Object(S3BUCKET_NAME, f"inbox/{from_date.strftime("%Y-%m-%d").replace('-','_')}_{to_date.strftime("%Y-%m-%d").replace('-','_')}_consumer_complaints.json")
    s3object.put(
        Body = (bytes(json.dumps(consumer_complaints_data).encode('UTF-8')))
    )

    ingest_from_date_to_date({"from_date": from_date_object, "to_date": to_date_object})
    return {
        'statusCode': 200,
        'body': json.dumps("New date Entries have been succefully writted to your mongoDB collection")
    }

In [25]:
TO_DATE = datetime.datetime.now()
TO_DATE.strftime("%Y-%m-%d")

'2023-06-06'